In [ ]:
import time
import pandas as pd
import hvplot.xarray
import matplotlib.pyplot as plt
from dask_gateway import Gateway
from dask.distributed import Client
from IPython.display import IFrame
from rich import print

plt.rcParams['figure.figsize'] = (15,10)
%matplotlib inline

In [ ]:
def summarize_inputs(recipe, ninputs=8901):
    """A helper function to use below
    """
    for time_index, url in recipe.file_pattern.items():
        if time_index[0] < 3 or time_index[0] > (ninputs-4):
            print(time_index, url)
        elif time_index[0] == (ninputs-4):
            print("...")
        else:
            pass

In [ ]:
# I'll start and connect to this cluster ahead of time, but time it, so you'll know how long it took!
start = time.time()

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
print(f"Connected to Dask client in {round(time.time()-start, 2)} seconds")
client

# Pangeo Forge: ETL for analysis-ready, cloud-optimized (ARCO) data stores

**Charles Stern** ([@cisaacstern](http://github.com/cisaacstern)), Data Infrastructure Engineer, Lamont-Doherty Earth Observatory (LDEO)

Presentation Repo: https://github.com/cisaacstern/zarr-vs-download

## CMEMS sea surface altimetry data

For this example we will use [gridded sea-surface altimetry data](http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047) from The Copernicus Marine Environment, a widely used dataset in physical oceanography and climate.

## CMEMS `ftp` index

```
ftp://my.cmems-du.eu:...
 ├──/1993
 │   ├──/01
 │   │   ├── dt_global...19930101.nc  (7789577 bytes)
 │   │  ...
 │   │   └── dt_global...19930131.nc  (7853172 bytes)
 │  ...
 │   └──/12
...
 └──/2020
```

In [ ]:
# Our target range includes 8901 files:

dates = pd.date_range(start="1993-01-01", end="2017-05-15")
len(dates)

### The old way: start downloading

The script `wget_cmems.py` will take care of this for us. Let's pause here and get it started.

# A better way: Pangeo Forge

`pangeo-forge-recipes` provides logic for transforming all of these source files into a single consolidated zarr store.

## What's a `recipe`?
A `recipe` is a Python file which can "see" all of the source files, and also knows how to logically arrange them into a cohesive dataset.

In [ ]:
from cmems_recipe import recipe

summarize_inputs(recipe)

These are the same files as we've begun downloading, with the addition of "alignment keys" (printed above in green) which allow `pangeo-forge-recipes` to **assemble** them into a cloud-optimized zarr store.

## Initialize Dataset

> In order to build a zarr store from source files, the recipe must be executed.

I've already completed this build, so from this point forward, getting up and running with the data is nearly **instantaneous**.

There are a few options for loading the dataset from the zarr store, but using an `intake` catalog is one of the easiest.

In [ ]:
from intake import open_catalog
cat = open_catalog("catalog.yaml")

for source in ["full_altimetry", "anomalies_only"]:
    ds = cat[source].to_dask()
    bold, reset = '\033[1m', '\033[0m'
    print(f"{bold}'{source}' is {round(ds.nbytes/1e9, 2)} GBs {reset}and contains {ds.data_vars} \n")

In [ ]:
ds

## Visually Examine Some of the Data

Let's do a sanity check that the data looks reasonable. Here we use the [hvplot](https://hvplot.holoviz.org/index.html) interactive plotting library.

In [ ]:
ds.sla.hvplot.image('longitude', 'latitude',
                    rasterize=True, dynamic=True, width=800, height=450,
                    widget_type='scrubber', widget_location='bottom', cmap='RdBu_r')

## Example calculation: timeseries of Global Mean Sea Level

Here we make a simple yet fundamental calculation: the rate of increase of global mean sea level over the observational period.

In [ ]:
# the number of GB involved in the reduction
ds.sla.nbytes/1e9

In [ ]:
IFrame(client.dashboard_link, width=900, height=550)

In [ ]:
# the computationally intensive step
sla_timeseries = ds.sla.mean(dim=('latitude', 'longitude')).load()

In [ ]:
sla_timeseries.plot(label='full data')
sla_timeseries.rolling(time=365, center=True).mean().plot(label='rolling annual mean')
plt.ylabel('Sea Level Anomaly [m]')
plt.title('Global Mean Sea Level')
plt.legend()
plt.grid()

### Remember the download we started with `wget_cmems.py`?
   - We just did a lot of work analyzing a 73 GB subset of a 500+ GB dataset.
   - How much of that same dataset has been downloaded over these last few minutes?
   - Let's see ...

1. How do I make my own recipe?
   - Check out the docs: https://pangeo-forge.readthedocs.io/en/latest/
   - Reach out by opening an Issue here: https://github.com/pangeo-forge/staged-recipes/issues.
   - One prerequisite is that your dataset is in a format that can be opened with `xarray` (netCDF, etc.).
2. What's next for `pangeo-forge-recipes`?
   - Automated recipe execution in "Bakeries"
   - Cataloging with STAC
3. Your questions!
   - Reach out on GitHub if you think of them later :-)

Feel free to complete the additional computations and plots below on your own time. They're pretty interesting!

## Additional computations: Hovöller diagram & standard deviation

Here are some additional computations to run on the Dask cluster.

In order to understand how the sea level rise is distributed in latitude, we can make a sort of [Hovmöller diagram](https://en.wikipedia.org/wiki/Hovm%C3%B6ller_diagram).

In [ ]:
# another distributed computation
sla_hov = ds.sla.mean(dim='longitude').load()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
sla_hov.name = 'Sea Level Anomaly [m]'
sla_hov.transpose().plot(vmax=0.2, ax=ax)

We can see that most sea level rise is actually in the Southern Hemisphere.

## Sea Level Variability

We can examine the natural variability in sea level by looking at its standard deviation in time.

In [ ]:
# one last computation
sla_std = ds.sla.std(dim='time').load()
sla_std.name = 'Sea Level Variability [m]'

In [ ]:
ax = sla_std.plot()
_ = plt.title('Sea Level Variability')